In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import gc
import math
from tqdm import tqdm
import csv
csv.field_size_limit(500 * 1024 * 1024)

131072

# 处理训练数据

## 处理静态文件以及操作数据

In [21]:
def prepare_operation():
    ad_operation = pd.read_table('./data/final_map_bid_opt.out',header=None,engine='python')
    ad_operation.rename(columns = {0:'ad_id',1:'modify_time',2:'opera_type',3:'ad_type',4:'billing_type',5:'new_bid'}, inplace = True)
    ad_operation.sort_values(by=['ad_id','modify_time'],inplace=True)
    ad_operation.reset_index(drop=True,inplace=True)
    ad_operation['day'] = ad_operation['modify_time'].apply(lambda x : int(str(x)[6:8]))
    ad_operation['hours'] = ad_operation['modify_time'].apply(lambda x : int(str(x)[8:10]))
    ad_operation.head()
    lastId = -1
    ad_operation_all = np.zeros((3107315,5),dtype=np.int)
    count = 0
    lastRow = None
    for index, row in tqdm(ad_operation.iterrows()):
        ad_id = row['ad_id']
        if ad_id != lastId and lastId == -1:
            for i in range(9,row['day']):
                ad_operation_all[count][:] = [ad_id,row['ad_type'],row['billing_type'],row['new_bid'],i]
                count += 1
        elif ad_id != lastId:
            for i in range(lastRow['day'],23):
                ad_operation_all[count][:] = [lastRow['ad_id'],lastRow['ad_type'],lastRow['billing_type'],lastRow['new_bid'],i]
                count += 1
            if row['opera_type'] == 1:
                for i in range(9,row['day']):
                    ad_operation_all[count][:] = [ad_id,row['ad_type'],row['billing_type'],row['new_bid'],i]
                    count += 1
        else:
            for i in range(lastRow['day'],row['day']):
                ad_operation_all[count][:] = [ad_id,lastRow['ad_type'],lastRow['billing_type'],lastRow['new_bid'],i]
                count += 1
        lastId = ad_id
        lastRow = row.copy()
    for i in range(lastRow['day'],23):
        ad_operation_all[count][:] = [lastRow['ad_id'],lastRow['ad_type'],lastRow['billing_type'],lastRow['new_bid'],i]
        count += 1
    print(count)
    ad_operation_all = pd.DataFrame(ad_operation_all[:count],columns=['ad_id','ad_type','billing_type', 'new_bid','day'])
    ad_operation_all.to_csv('./data/ad_operation_by_day.csv',index=0,header=True)

In [22]:
# 静态数据   ad_static_feature_csv.csv
print('pre-process ad_static_feature')
ad_static_feature = pd.read_table('./data/map_ad_static.out',header=None)
ad_static_feature.rename(columns = {0:'ad_id',1:'create_time',2:'ad_account_id',3:'commodity_id',4:'commodity_type',5:'ad_industry_id',6:'material_size'}, inplace = True)
ad_static_feature.to_csv('./data/ad_static_feature.csv',index=0,header=True)
# 广告操作数据  ad_operation_csv.csv
print('pre-process ad_operation')
prepare_operation()
# 用户数据  user_data_csv.csv
print('pre-process user_data')
user_data = pd.read_table('./data/user_data.out',header=None,engine='python')
user_data.rename(columns={0:'user_id',1:'age',2:'gender',3:'area',4:'status',5:'education',6:'consuptionAbility',7:'device',8:'work',9:'connectionType',10:'behavior'}, inplace = True)
user_data.to_csv('./data/user_data.csv',index=0,header=True)

pre-process ad_operation


321753it [01:55, 2788.48it/s]


3107314


## 构建训练数据

In [2]:
# 统计广告的转化率
train_log_time = ['0410','0411','0412','0413','0414','0415','0416','0417','0418','0419','0420','0421','0422']

for _time in train_log_time:
    print('curr time : 2019',_time)
    aid_dict = {}           # key ad_id  value=[total,pos]
    path = './data/track_log/track_log_2019' + _time + '.out'
    exposureLog = pd.read_table(path,header=None)
    for index, row in tqdm(exposureLog.iterrows()):
        user_id = row[2]
        ad_info = row[4].split(';')
        for i in range(len(ad_info)):
            ad_split_info = ad_info[i].split(',')
            ad_id = int(ad_split_info[0])
            if ad_id not in aid_dict.keys():
                aid_dict[ad_id] = [0,0]
            if ad_split_info[6] == '1':
                aid_dict[ad_id][0] += 1
                aid_dict[ad_id][1] += 1
            else:
                aid_dict[ad_id][0] += 1
    tmp=pd.DataFrame.from_dict(data=aid_dict, orient='index')
    tmp.columns = ['total_aid', 'pos_aid']
    tmp.reset_index(inplace = True)
    tmp.rename(columns={'index': 'ad_id'},inplace = True)
    outName = 'aidCount' + _time + '.csv'
    tmp.to_csv(outName,index=0,header=True)

curr time : 2019 0410


283835it [01:10, 4044.89it/s]


curr time : 2019 0411


265890it [01:04, 4146.26it/s]


curr time : 2019 0412


...

462172it [01:51, 4155.16it/s]


curr time : 2019 0421


478730it [01:57, 4066.23it/s]


curr time : 2019 0422


348519it [01:27, 3964.77it/s]


In [4]:
#统计用户定向长度特征
user_data = pd.read_csv('./user_data.csv')
uid_age_dict = {}               #key user_id    value  age
uid_gender_dict = {}            #key user_id    value  gender
uid_education_dict = {}         #key user_id    value  education
uid_consuptionAbility_dict = {} #key user_id    value  consuptionAbility
uid_device_dict = {}            #key user_id    value  device
uid_connectionType_dict = {}    #key user_id    value  connectionType

uid_work_dict = {}              #key user_id    value   [] work
uid_area_dict = {}              #key user_id    value   [] area
uid_status_dict = {}            #key user_id    value   [] status
uid_behavior_dict = {}          #key user_id    value   [] behavior
single_feature = ['age','gender','education','consuptionAbility','device','connectionType']
multi_feature = ['work','area','status','behavior']

for index, row in tqdm(user_data.iterrows()):
    uid = row['user_id']
    uid_age_dict[uid] = row['age']
    uid_gender_dict[uid] = row['gender']
    uid_education_dict[uid] = row['education']
    uid_consuptionAbility_dict[uid] = row['consuptionAbility']
    uid_device_dict[uid] = row['device']
    uid_connectionType_dict[uid] = row['connectionType']
    
    uid_work_dict[uid] = row['work'].split(',')
    uid_area_dict[uid] = row['area'].split(',')
    uid_status_dict[uid] = row['status'].split(',')

train_log_time = ['0410','0411','0412','0413','0414','0415','0416','0417','0418','0419','0420','0421','0422']
count = 0
for _time in train_log_time:
    print('curr time : 2019',_time)
    aid_age_dict = {}               #key user_id    value  len age
    aid_gender_dict = {}            #key user_id    value  len gender
    aid_education_dict = {}         #key user_id    value  len education
    aid_consuptionAbility_dict = {} #key user_id    value  len consuptionAbility
    aid_device_dict = {}            #key user_id    value  len device
    aid_connectionType_dict = {}    #key user_id    value  len connectionType
    
    aid_work_dict = {}              #key user_id    value   [] len work
    aid_area_dict = {}              #key user_id    value   [] len area
    aid_status_dict = {}            #key user_id    value   [] len status
    
    path = '../data/track_log/track_log_2019' + _time + '.out'
    exposureLog = pd.read_table(path,header=None)
    for index, row in tqdm(exposureLog.iterrows()):
        user_id = row[2]
        if user_id not in uid_age_dict.keys():
            count += 1
            continue
        age = uid_age_dict[user_id]
        gender = uid_gender_dict[user_id]
        education = uid_education_dict[user_id]
        consuptionAbility = uid_consuptionAbility_dict[user_id]
        device = uid_device_dict[user_id]
        connectionType = uid_connectionType_dict[user_id]
        
        work = uid_work_dict[user_id]
        area = uid_area_dict[user_id]
        status = uid_status_dict[user_id]
        
        ad_info = row[4].split(';')
        for i in range(len(ad_info)):
            ad_split_info = ad_info[i].split(',')
            ad_id = int(ad_split_info[0])
            if ad_id not in aid_age_dict.keys():
                aid_age_dict[ad_id] = set()
                aid_gender_dict[ad_id] = set()
                aid_education_dict[ad_id] = set()
                aid_consuptionAbility_dict[ad_id] = set()
                aid_device_dict[ad_id] = set()
                aid_connectionType_dict[ad_id] = set()
                aid_work_dict[ad_id] = set()
                aid_area_dict[ad_id] = set()
                aid_status_dict[ad_id] = set()
            aid_age_dict[ad_id].add(age)
            aid_gender_dict[ad_id].add(gender)
            aid_education_dict[ad_id].add(education)
            aid_consuptionAbility_dict[ad_id].add(consuptionAbility)
            aid_device_dict[ad_id].add(device)
            aid_connectionType_dict[ad_id].add(connectionType)

            for fea in work:
                aid_work_dict[ad_id].add(fea)
            for fea in area:
                aid_area_dict[ad_id].add(fea)
            for fea in status:
                aid_status_dict[ad_id].add(fea)
    
    uid_cnt_info = np.zeros((500000,10),dtype=int)
    cnt = 0
    for ad_id,v in aid_age_dict.items():
        uid_cnt_info[cnt][:] = [ad_id,len(aid_age_dict[ad_id]),len(aid_gender_dict[ad_id]),len(aid_education_dict[ad_id]),
                               len(aid_consuptionAbility_dict[ad_id]),len(aid_device_dict[ad_id]),len(aid_connectionType_dict[ad_id]),
                               len(aid_work_dict[ad_id]),len(aid_area_dict[ad_id]),len(aid_status_dict[ad_id])]
        cnt += 1
    tmp=pd.DataFrame(uid_cnt_info[:cnt],columns=['ad_id','len_age','len_gender','len_education',
                                                'len_consuptionAbility','len_device','len_connectionType','len_work',
                                                'len_area','len_status'])
    outName = './data/uidCount' + _time + '.csv'
    tmp.to_csv(outName,index=0,header=True)

In [5]:
def gen_train_and_target():
    train_log_time = ['0410','0411','0412','0413','0414','0415','0416','0417','0418','0419','0420','0421','0422']
    week_dict = {10:2,11:3,12:4,13:5,14:6,15:0,16:1,17:2,18:3,19:4,20:5,21:6,22:0}
    train_df =  pd.DataFrame()
    for _time in train_log_time:
        outName = './data/aidCount' + _time + '.csv'
        tmp = pd.read_csv(outName)
        tmp['day'] = int(_time[2:4])
        outName = './data/uidCount' + _time + '.csv'
        tmp1 = pd.read_csv(outName)
        tmp = pd.merge(tmp,tmp1,on='ad_id',how='left')
        train_df = pd.concat([train_df, tmp], axis=0)
    train_df.rename(columns={'pos_aid': 'target'},inplace = True)
    train_df = pd.merge(train_df,pd.read_csv('./data/ad_static_feature.csv'),on='ad_id',how='left')
    train_df = pd.merge(train_df,pd.read_csv('./data/ad_operation_by_day.csv'),on=['ad_id','day'],how='left')
    train_df.fillna(-3,inplace=True)
    print(train_df.shape)
    ## 操作数据中找不到的全部去掉
    train_df = train_df[train_df['billing_type'] >= -1]
    train_df = train_df[train_df['ad_account_id'] >= -1]
    train_df = train_df[train_df['commodity_type'] >= -1]
    train_df = train_df[train_df['ad_industry_id'] >= -1]
    train_df = train_df[train_df['material_size'] >= -1]
    train_df = train_df[train_df['ad_type'] >= -1]
    print(train_df.shape)
    train_df.sort_values(by=['ad_id','day'],inplace=True)
    train_df.drop('create_time',axis=1,inplace = True)
    train_df.reset_index(drop=True,inplace = True)
    train_df = train_df.astype(np.int)
    train_df['weekday'] = train_df['day'].apply(lambda x: week_dict[x])
    return train_df
train_df = gen_train_and_target()
print(train_df.shape)
train_df.head()
train_df.to_csv('./data/train_df_v1.csv',index=0,header=True)

(775360, 22)
(735118, 22)
(735118, 22)


In [17]:
def gen_train_pos_aid_fea_by_day(gen_dev=False):
    train_data = pd.read_csv('./data/train_df_v1.csv')
    train_data.fillna(-1,inplace=True)
    if gen_dev:
        dev_data = train_data[train_data['day'] == 22]
        train_data = train_data[train_data['day'] < 22]
        dev_data.reset_index(inplace=True,drop=True,)
        train_data.reset_index(inplace=True,drop=True,)
    train_data['aid_convert_rate'] = train_data.apply(lambda x : (1 + x['target']) / (1200 + x['total_aid']),axis=1)
    train_data['aid_convert_rate'] = train_data['aid_convert_rate'].round(decimals = 5)
    # count all aid convert
    test1 = train_data.groupby('ad_id')['target'].sum()
    test1 = test1.to_frame()
    test1.reset_index(inplace=True)
    test1.rename(columns={'target':'target_t'},inplace=True)
    test2 = train_data.groupby('ad_id')['total_aid'].sum()
    test2 = test2.to_frame()
    test2.reset_index(inplace=True)
    test2.rename(columns={'total_aid':'total_aid_t'},inplace=True)
    train_data = pd.merge(train_data,test1,on='ad_id',how='left')
    train_data = pd.merge(train_data,test2,on='ad_id',how='left')
    train_data['aid_convert_rate_t'] = train_data.apply(lambda x : (x['target_t']) / (x['total_aid_t']),axis=1)
    train_data['aid_convert_rate_t'] = train_data['aid_convert_rate_t'].round(decimals = 5)
    train_data.drop(['target_t','total_aid_t'],axis=1,inplace=True)
    count_feature_list = ['ad_account_id','commodity_id','ad_industry_id','material_size','ad_type']
    for fea in count_feature_list:      
        test1 = train_data.groupby(fea)['target'].sum()
        test1 = test1.to_frame()
        test1.reset_index(inplace=True)
        test1.rename(columns={'target': fea + '_target'},inplace=True)
        test2 = train_data.groupby(fea)['total_aid'].sum()
        test2 = test2.to_frame()
        test2.reset_index(inplace=True)
        test2.rename(columns={'total_aid': fea + '_aid'},inplace=True)
        train_data = pd.merge(train_data,test1,on=fea,how='left')
        train_data = pd.merge(train_data,test2,on=fea,how='left')
        if fea == 'ad_account_id' or fea == 'commodity_id' or fea == 'ad_industry_id':
            train_data[fea + '_convert_rate'] = train_data.apply(lambda x : (x[fea + '_target']) / (x[fea + '_aid']),axis=1)
        else:
            train_data[fea + '_convert_rate'] = train_data.apply(lambda x : x[fea + '_target'] / x[fea + '_aid'],axis=1)
        train_data[fea + '_convert_rate'] = train_data[fea + '_convert_rate'].round(decimals = 5)
        print('train',fea,'done')
    train_data = train_data.sort_values(by=['ad_id','day'])
    train_data.reset_index(inplace=True,drop=True)
    if gen_dev:
        # 避免数据穿越 使用21号之前的训练数据构建
        tmp_data = train_data.groupby('ad_id')['aid_convert_rate'].last().reset_index()
        aid_convert_rate_median = tmp_data['aid_convert_rate'].sort_values().reset_index(drop=True)[round(tmp_data.shape[0]/2)]
        dev_data = pd.merge(dev_data,tmp_data,on='ad_id',how='left')
        dev_data['aid_convert_rate'].fillna(aid_convert_rate_median,inplace=True)         
        tmp_data = train_data.groupby('ad_id')['aid_convert_rate_t'].last().reset_index()
        aid_convert_rate_t_median = tmp_data['aid_convert_rate_t'].sort_values().reset_index(drop=True)[round(tmp_data.shape[0]/2)]
        dev_data = pd.merge(dev_data,tmp_data,on='ad_id',how='left')
        dev_data['aid_convert_rate_t'].fillna(aid_convert_rate_t_median,inplace=True)  
        # count by day
        for fea in count_feature_list:
            tmp_data = train_data.groupby(fea)[fea+'_convert_rate'].last().reset_index()
            print(fea,tmp_data.shape)
            median_convrert_rate = tmp_data[fea+'_convert_rate'].sort_values().reset_index(drop=True)[round(tmp_data.shape[0]/2)]
            dev_data = pd.merge(dev_data,tmp_data,on=fea,how='left')
            dev_data[fea+'_convert_rate'].fillna(median_convrert_rate,inplace=True)  
            print('dev',fea,'done')
    if gen_dev:
        unkown_feature = ['ad_account_id',
           'ad_industry_id', 'ad_type',
           'billing_type', 'commodity_id', 
           'commodity_type', 'material_size']
        for fea in unkown_feature:
            fea_set = set(train_data[fea])
            mode = train_data[fea].mode().loc[0]
            print(fea,mode)
            dev_data[fea] = dev_data[fea].apply(lambda x : x if x in fea_set else mode)
        
    for fea in count_feature_list:
        train_data.drop([fea + '_target',fea + '_aid'],axis=1,inplace=True)
    if gen_dev:
        train_data.to_csv('./data/train_df_v1_10_21.csv',index=False,header=True)
        dev_data.to_csv('./data/train_df_v1_22.csv',index=False,header=True)
    else:
        train_data.to_csv('./data/train_df_v1_10_22.csv',index=False,header=True)
    return train_data

In [18]:
train = gen_train_pos_aid_fea_by_day(gen_dev=True)
train = gen_train_pos_aid_fea_by_day(gen_dev=False)

train ad_account_id done
train commodity_id done
train ad_industry_id done
train material_size done
train ad_type done
ad_account_id (11083, 2)
dev ad_account_id done
commodity_id (6801, 2)
dev commodity_id done
ad_industry_id (195, 2)
dev ad_industry_id done
material_size (19, 2)
dev material_size done
ad_type (15, 2)
dev ad_type done
ad_account_id 11882
ad_industry_id 187
ad_type 5
billing_type 2
commodity_id -1
commodity_type 4
material_size 7
train ad_account_id done
train commodity_id done
train ad_industry_id done
train material_size done
train ad_type done


# 处理测试数据

In [19]:
train = pd.read_csv('./data/train_df_v1_10_22.csv')

In [21]:
final_select_test_request = pd.read_table('./data/Btest_select_request_20190424.out',header=None,engine='python')
test_sample_request = np.zeros((300000*250,3),dtype=int)
count = 0
for index, row in tqdm(final_select_test_request.iterrows()):
    ad_id = row[0]
    ad_request_info = row[1].split('|')
    for s in ad_request_info:
        request_id = s.split(',')[0]
        ad_space_id = s.split(',')[1]
        test_sample_request[count][:] = [ad_id,request_id,ad_space_id]
        count += 1
test_request_df = pd.DataFrame(test_sample_request[:count],columns=['ad_id','request_id','ad_space_id'])

aid_age_dict = {}               #key user_id    value  len age
aid_gender_dict = {}            #key user_id    value  len gender
aid_education_dict = {}         #key user_id    value  len education
aid_consuptionAbility_dict = {} #key user_id    value  len consuptionAbility
aid_device_dict = {}            #key user_id    value  len device
aid_connectionType_dict = {}    #key user_id    value  len connectionType
aid_work_dict = {}              #key user_id    value   [] len work
aid_area_dict = {}              #key user_id    value   [] len area
aid_status_dict = {}            #key user_id    value   [] len status

BTest_tracklog = pd.read_table('./data/BTest_tracklog_20190424.txt',header=None,engine='python')
BTest_tracklog = BTest_tracklog.drop(4,axis=1)
BTest_tracklog.rename(columns = {0:'request_id',1:'request_time',2:'user_id',3:'ad_space_id'},inplace = True)
test_log = pd.merge(test_request_df,BTest_tracklog,on=['request_id','ad_space_id'],how='left')
for index, row in tqdm(test_log.iterrows()):
    user_id = row['user_id']
    ad_id = row['ad_id']
    if user_id not in uid_age_dict.keys():
        continue
    age = uid_age_dict[user_id]
    gender = uid_gender_dict[user_id]
    education = uid_education_dict[user_id]
    consuptionAbility = uid_consuptionAbility_dict[user_id]
    device = uid_device_dict[user_id]
    connectionType = uid_connectionType_dict[user_id]
    work = uid_work_dict[user_id]
    area = uid_area_dict[user_id]
    status = uid_status_dict[user_id]
    if ad_id not in aid_age_dict.keys():
        aid_age_dict[ad_id] = set()
        aid_gender_dict[ad_id] = set()
        aid_education_dict[ad_id] = set()
        aid_consuptionAbility_dict[ad_id] = set()
        aid_device_dict[ad_id] = set()
        aid_connectionType_dict[ad_id] = set()
        aid_work_dict[ad_id] = set()
        aid_area_dict[ad_id] = set()
        aid_status_dict[ad_id] = set()
    aid_age_dict[ad_id].add(age)
    aid_gender_dict[ad_id].add(gender)
    aid_education_dict[ad_id].add(education)
    aid_consuptionAbility_dict[ad_id].add(consuptionAbility)
    aid_device_dict[ad_id].add(device)
    aid_connectionType_dict[ad_id].add(connectionType)
    for fea in work:
        aid_work_dict[ad_id].add(fea)
    for fea in area:
        aid_area_dict[ad_id].add(fea)
    for fea in status:
        aid_status_dict[ad_id].add(fea)
uid_cnt_info = np.zeros((500000,10),dtype=int)
cnt = 0
for ad_id,v in aid_age_dict.items():
    uid_cnt_info[cnt][:] = [ad_id,len(aid_age_dict[ad_id]),len(aid_gender_dict[ad_id]),len(aid_education_dict[ad_id]),
                            len(aid_consuptionAbility_dict[ad_id]),len(aid_device_dict[ad_id]),len(aid_connectionType_dict[ad_id]),
                            len(aid_work_dict[ad_id]),len(aid_area_dict[ad_id]),len(aid_status_dict[ad_id])]
    cnt += 1
tmp=pd.DataFrame(uid_cnt_info[:cnt],columns=['ad_id','len_age','len_gender','len_education',
                                            'len_consuptionAbility','len_device','len_connectionType','len_work',
                                            'len_area','len_status'])
outName = './data/uidCountTest.csv'
tmp.to_csv(outName,index=0,header=True)

In [23]:
test_sample_bid = pd.read_table('./data/Btest_sample_bid.out',header=None,engine='python')
test_sample_bid.rename(columns = {0:'sample_id',1:'ad_id',2:'ad_type',3:'billing_type',4:'new_bid'}, inplace = True)
test_sample_bid['day'] = 24
test_sample_bid['weekday'] = 2
print(test_sample_bid.shape)

final_select_test_request = pd.read_table('./data/Btest_select_request_20190424.out',header=None,engine='python')
test_sample_request = np.zeros((300000*250,3),dtype=int)
count = 0
for index, row in (final_select_test_request.iterrows()):
    ad_id = row[0]
    ad_request_info = row[1].split('|')
    for s in ad_request_info:
        request_id = s.split(',')[0]
        ad_space_id = s.split(',')[1]
        test_sample_request[count][:] = [ad_id,request_id,ad_space_id]
        count += 1
print(count)
test_request_df = pd.DataFrame(test_sample_request[:count],columns=['ad_id','request_id','ad_space_id'])
test2 = test_request_df.groupby('ad_id')['request_id'].count()
test2 = test2.to_frame()
test2.reset_index(inplace=True)
test2.rename(columns={'request_id':'total_aid'},inplace=True)
print(test2.shape)
test_df = pd.merge(test_sample_bid,test2,on='ad_id',how='left')
print(test_df.shape)
test_df.head()
test_df = pd.merge(test_df,pd.read_csv('./data/ad_static_feature.csv'),on='ad_id',how='left');
test_df = pd.merge(test_df,pd.read_csv('./data/uidCountTest.csv'),on='ad_id',how='left');
print(test_df.shape)
test_df.head()

(62968, 7)



1617it [00:30, 313.39it/s]

11484349
(5740, 2)
(62968, 8)
(62968, 23)


,sample_id,ad_id,ad_type,billing_type,new_bid,day,weekday,total_aid,create_time,ad_account_id,...,material_size,len_age,len_gender,len_education,len_consuptionAbility,len_device,len_connectionType,len_work,len_area,len_status
0,1,15,5,1,432,24,2,1757,1555579069,10125,...,8,56,3,7,3,3,5,7,4016,16
1,2,15,5,1,548,24,2,1757,1555579069,10125,...,8,56,3,7,3,3,5,7,4016,16
2,3,15,5,1,557,24,2,1757,1555579069,10125,...,8,56,3,7,3,3,5,7,4016,16
3,4,15,5,1,706,24,2,1757,1555579069,10125,...,8,56,3,7,3,3,5,7,4016,16
4,5,15,5,1,1132,24,2,1757,1555579069,10125,...,8,56,3,7,3,3,5,7,4016,16


In [24]:
# 构造测试集的真实出价   中位数填充
test_bid = test_df.groupby(['ad_id'])['new_bid'].median().reset_index()
test_df.drop('new_bid',axis=1,inplace=True)
test_df = pd.merge(test_df,test_bid,on='ad_id',how='left')    
test_df['new_bid'] = test_df['new_bid'].astype(np.int)
print(test_df.shape)
test_df.head(3)

(62968, 23)


,sample_id,ad_id,ad_type,billing_type,day,weekday,total_aid,create_time,ad_account_id,commodity_id,...,len_age,len_gender,len_education,len_consuptionAbility,len_device,len_connectionType,len_work,len_area,len_status,new_bid
0,1,15,5,1,24,2,1757,1555579069,10125,6524,...,56,3,7,3,3,5,7,4016,16,1370
1,2,15,5,1,24,2,1757,1555579069,10125,6524,...,56,3,7,3,3,5,7,4016,16,1370
2,3,15,5,1,24,2,1757,1555579069,10125,6524,...,56,3,7,3,3,5,7,4016,16,1370


In [25]:
def gen_test_pos_aid_fea_by_day(test_df,train_data):    
    count_feature_list = ['ad_account_id','commodity_id','ad_industry_id','material_size','ad_type']
    tmp_data = train_data.groupby('ad_id')['aid_convert_rate'].last().reset_index()
    test_df = pd.merge(test_df,tmp_data,on='ad_id',how='left')
    tmp_data = train_data.groupby('ad_id')['aid_convert_rate_t'].last().reset_index()
    test_df = pd.merge(test_df,tmp_data,on='ad_id',how='left')
    
    aid_convert_rate_median = tmp_data['aid_convert_rate_t'].sort_values().reset_index(drop=True)[round(tmp_data.shape[0]/2)]
    print('aid_convert_rate_median',aid_convert_rate_median)
    test_df['aid_convert_rate_t'].fillna(aid_convert_rate_median,inplace=True)  
    test_df['aid_convert_rate'].fillna(aid_convert_rate_median,inplace=True)  

    for fea in count_feature_list:
        tmp_data = train_data.groupby(fea)[fea+'_convert_rate'].last().reset_index()
        median_convrert_rate = tmp_data[fea+'_convert_rate'].sort_values().reset_index(drop=True)[round(tmp_data.shape[0]/2)]
        print('median_convrert_rate',median_convrert_rate)
        test_df = pd.merge(test_df,tmp_data,on=fea,how='left')
        test_df[fea+'_convert_rate'].fillna(median_convrert_rate,inplace=True)  
    test_df.drop(['create_time'],axis=1,inplace=True)
    return test_df

In [26]:
df1=test_df.copy()
df2=train.copy()
test = gen_test_pos_aid_fea_by_day(df1,df2)

aid_convert_rate_median 0.00065
median_convrert_rate 0.00686
median_convrert_rate 0.0012699999999999999
median_convrert_rate 0.011859999999999999
median_convrert_rate 0.01697
median_convrert_rate 0.013080000000000001


In [27]:
test.to_csv('./data/testB_df_v1.csv',index=0,header=True)

# 添加其他特征

通过特征重要性分析，我们对目标转化率和目标曝光值这两个重要特征进行进一步建模

In [28]:
def get_other_fea(train,test):
    test_df = test.copy()
    train_df = train.copy()
    target_max = train_df.groupby('ad_id')['target'].max()
    target_max = target_max.to_frame()
    target_max.reset_index(inplace=True)
    target_max.rename(columns={'target':'target_max'},inplace=True)
    target_min = train_df.groupby('ad_id')['target'].min()
    target_min = target_min.to_frame()
    target_min.reset_index(inplace=True)
    target_min.rename(columns={'target':'target_min'},inplace=True)
    target_mean = train_df.groupby('ad_id')['target'].mean()
    target_mean = target_mean.to_frame()
    target_mean.reset_index(inplace=True)
    target_mean.rename(columns={'target':'target_mean'},inplace=True)
    target_median = train_df.groupby('ad_id')['target'].median()
    target_median = target_median.to_frame()
    target_median.reset_index(inplace=True)
    target_median.rename(columns={'target':'target_median'},inplace=True)
    
    aid_convert_rate_max = train_df.groupby('ad_id')['aid_convert_rate'].max()
    aid_convert_rate_max = aid_convert_rate_max.to_frame()
    aid_convert_rate_max.reset_index(inplace=True)
    aid_convert_rate_max.rename(columns={'aid_convert_rate':'aid_convert_rate_max'},inplace=True)
    aid_convert_rate_min = train_df.groupby('ad_id')['aid_convert_rate'].min()
    aid_convert_rate_min = aid_convert_rate_min.to_frame()
    aid_convert_rate_min.reset_index(inplace=True)
    aid_convert_rate_min.rename(columns={'aid_convert_rate':'aid_convert_rate_min'},inplace=True)
    aid_convert_rate_mean = train_df.groupby('ad_id')['aid_convert_rate'].mean()
    aid_convert_rate_mean = aid_convert_rate_mean.to_frame()
    aid_convert_rate_mean.reset_index(inplace=True)
    aid_convert_rate_mean.rename(columns={'aid_convert_rate':'aid_convert_rate_mean'},inplace=True)
    aid_convert_rate_median = train_df.groupby('ad_id')['aid_convert_rate'].median()
    aid_convert_rate_median = aid_convert_rate_median.to_frame()
    aid_convert_rate_median.reset_index(inplace=True)
    aid_convert_rate_median.rename(columns={'aid_convert_rate':'aid_convert_rate_median'},inplace=True)
    
    train_df = pd.merge(train_df,target_max,on='ad_id',how='left')
    test_df = pd.merge(test_df,target_max,on='ad_id',how='left')
    train_df = pd.merge(train_df,target_min,on='ad_id',how='left')
    test_df = pd.merge(test_df,target_min,on='ad_id',how='left')
    train_df = pd.merge(train_df,target_mean,on='ad_id',how='left')
    test_df = pd.merge(test_df,target_mean,on='ad_id',how='left')
    train_df = pd.merge(train_df,target_median,on='ad_id',how='left')
    test_df = pd.merge(test_df,target_median,on='ad_id',how='left')
    train_df = pd.merge(train_df,aid_convert_rate_max,on='ad_id',how='left')
    test_df = pd.merge(test_df,aid_convert_rate_max,on='ad_id',how='left')
    train_df = pd.merge(train_df,aid_convert_rate_min,on='ad_id',how='left')
    test_df = pd.merge(test_df,aid_convert_rate_min,on='ad_id',how='left')
    train_df = pd.merge(train_df,aid_convert_rate_mean,on='ad_id',how='left')
    test_df = pd.merge(test_df,aid_convert_rate_mean,on='ad_id',how='left')
    train_df = pd.merge(train_df,aid_convert_rate_median,on='ad_id',how='left')
    test_df = pd.merge(test_df,aid_convert_rate_median,on='ad_id',how='left')
    
    test_df.fillna(-1,inplace=True)  
    train_df.fillna(-1,inplace=True)
    
    return train_df,test_df

In [29]:
test_df = pd.read_csv('./data/testB_df_v1.csv')
train_df1 = pd.read_csv('./data/train_df_v1_10_22.csv')
train_df2 = pd.read_csv('./data/train_df_v1_10_21.csv')
dev_df = pd.read_csv('./data/train_df_v1_22.csv')
train_df2,dev_df = get_other_fea(train_df2,dev_df)
train_df1,test_df = get_other_fea(train_df1,test_df)

In [30]:
test_df.to_csv('./data/testB_df_v1_1.csv',index=0,header=True)
train_df1.to_csv('./data/train_df_v1_10_22_1.csv',index=0,header=True)
train_df2.to_csv('./data/train_df_v1_10_21_1.csv',index=0,header=True)
dev_df.to_csv('./data/train_df_v1_22_1.csv',index=0,header=True)

# 特征离散化

In [31]:
# 特征离散化
test_df = pd.read_csv('./data/testB_df_v1_1.csv')
train_df1 = pd.read_csv('./data/train_df_v1_10_22_1.csv')
train_df2 = pd.read_csv('./data/train_df_v1_10_21_1.csv')
dev_df = pd.read_csv('./data/train_df_v1_22_1.csv')
print(test_df.shape,train_df1.shape,train_df2.shape,dev_df.shape)

(62968, 37) (735118, 37) (677738, 37) (57380, 37)


In [37]:
digit_features = ['ad_account_id_convert_rate', 
        'ad_industry_id_convert_rate',
        'ad_type_convert_rate',
        'aid_convert_rate',
        'aid_convert_rate_t',
        'commodity_id_convert_rate', 
        'material_size_convert_rate',
        'new_bid', 
        'total_aid', 
        'len_area',
        'aid_convert_rate_max',
        'aid_convert_rate_mean', 'aid_convert_rate_median',
        'aid_convert_rate_min', 'target_max', 'target_mean', 'target_median', 'target_min']

In [39]:
for fea in digit_features:
    print(fea,len(set(train_df1[fea])))

ad_account_id_convert_rate 3870
ad_industry_id_convert_rate 185
ad_type_convert_rate 15
aid_convert_rate 6605
aid_convert_rate_t 9738
commodity_id_convert_rate 1681
material_size_convert_rate 19
new_bid 9798
total_aid 20242
len_area 10685
aid_convert_rate_max 4777
aid_convert_rate_mean 26414
aid_convert_rate_median 5659
aid_convert_rate_min 2284
target_max 758
target_mean 3228
target_median 566
target_min 223


In [42]:
# 连续特征离散化
def digitize(test_df,train_df1,train_df2,dev_df,digit_features):
    test_df_this = pd.DataFrame(test_df,columns=digit_features)
    train_df1_this = pd.DataFrame(train_df1,columns=digit_features)
    train_df2_this = pd.DataFrame(train_df2,columns=digit_features)
    dev_df_this = pd.DataFrame(dev_df,columns=digit_features)
    total_df = pd.concat([test_df_this,train_df1_this,train_df2_this,dev_df_this], axis=0)
    for fea in digit_features:
        if fea=='ad_industry_id_convert_rate':  #183
            bins = [round(train_df1[fea].quantile(x),5) for x in (np.linspace( 0, 1, 20))]  
        elif fea=='commodity_id_convert_rate':  #1688  material_size
            bins = [round(train_df1[fea].quantile(x),5) for x in (np.linspace( 0, 1, 80))]
        elif fea=='material_size_convert_rate': #18  
            bins = [round(train_df1[fea].quantile(x),5) for x in (np.linspace( 0, 1, 6))]
        elif fea=='ad_type_convert_rate':       #13  
            bins = [round(train_df1[fea].quantile(x),5) for x in (np.linspace( 0, 1, 5))]
        else:
            bins = [round(train_df1[fea].quantile(x),5) for x in (np.linspace( 0, 1, 100))]
        bins.append(float(total_df[fea].max()+1))
        news_bins = []
        news_bins=list(set(bins))
        news_bins.sort(key=bins.index)
        news_bins.insert(0, float(total_df[fea].min()-1))
        print(fea,len(bins),len(news_bins))
        data = total_df[fea].values
        tmp = pd.cut(data,news_bins)
        total_df[fea] = tmp.codes
    test_df_this = total_df[:test_df.shape[0]]
    train_df1_this = total_df[test_df.shape[0]:test_df.shape[0]+train_df1.shape[0]]
    train_df2_this = total_df[test_df.shape[0]+train_df1.shape[0]:test_df.shape[0]+train_df1.shape[0]+train_df2.shape[0]]
    dev_df_this = total_df[test_df.shape[0]+train_df1.shape[0]+train_df2.shape[0]:]
    for fea in digit_features:
        test_df[fea] = test_df_this[fea]
        train_df1[fea] = train_df1_this[fea]
        train_df2[fea] = train_df2_this[fea]
        dev_df[fea] = dev_df_this[fea]
    #测试集未知特征众数填充
    unkown_feature = ['ad_account_id',
       'ad_industry_id', 'ad_type',
       'billing_type', 'commodity_id', 
       'commodity_type', 'material_size']
    for fea in unkown_feature:
        fea_set = set(train_df1[fea])
        mode = train_df1[fea].mode().loc[0]
        print(fea,mode)
        test_df[fea] = test_df[fea].apply(lambda x : x if x in fea_set else mode)
        
    test_df.to_csv('./data/test_df_v1_digit.csv', index=False,header=True)
    train_df1.to_csv('./data/train_df_v1_10_22_digit.csv', index=False,header=True)
    train_df2.to_csv('./data/train_df_v1_10_21_digit.csv', index=False,header=True)
    dev_df.to_csv('./data/train_df_v1_22_digit.csv', index=False,header=True)

In [43]:
digitize(test_df.copy(),train_df1.copy(),train_df2.copy(),dev_df.copy(),digit_features)

ad_account_id_convert_rate 101 100
ad_industry_id_convert_rate 21 20
ad_type_convert_rate 6 6
aid_convert_rate 101 61
aid_convert_rate_t 101 67
commodity_id_convert_rate 81 56
material_size_convert_rate 7 6
new_bid 101 99
total_aid 101 84
len_area 101 99
aid_convert_rate_max 101 71
aid_convert_rate_mean 101 80
aid_convert_rate_median 101 60
aid_convert_rate_min 101 60
target_max 101 28
target_mean 101 56
target_median 101 22
target_min 101 11
ad_account_id 11882
ad_industry_id 187
ad_type 5
billing_type 2
commodity_id -1
commodity_type 4
material_size 7
